In [30]:
import pandas as pd
import csv 

In [31]:
### EPA Air Toxics Screen Data Cleaneing ###

# function to read and add 'Year' column
def read_and_add_year(file_path, year):
    df_cancer = pd.read_csv(file_path + f'/{year}_Cancer_Risk_in_a_million_and_Noncancer_Risk_hazard_quotient.csv')
    df_cancer.insert(0, 'Year', year)
    return df_cancer

# Define file path
file_path = 'data/epa-airtoxscreen'

# Read and add 'Year' column
cancer_2014 = read_and_add_year(file_path, 2014)
cancer_2017 = read_and_add_year(file_path, 2017)
cancer_2018 = read_and_add_year(file_path, 2018)
cancer_2019 = read_and_add_year(file_path, 2019)

# function to remove rows with total cancer risk less than 0.01
def cancer_rows(*cancer_dfs):
    return [df[df['Total Cancer Risk (per million)'] >= 1] for df in cancer_dfs]

# Remove rows
cancer_2014, cancer_2017, cancer_2018, cancer_2019 = cancer_rows(cancer_2014, cancer_2017, cancer_2018, cancer_2019)

# Concatenate DataFrames for all years
epa_combined = pd.concat([cancer_2014, cancer_2017, cancer_2018, cancer_2019], ignore_index=True)

# format columns
epa_combined = epa_combined.drop(columns=['EPA Region','FIPS'])
epa_combined = epa_combined.rename(columns={'Tract': 'CensusTract'})
epa_combined = epa_combined[epa_combined['County'] != 'Entire State']
epa_combined = epa_combined.loc[:, epa_combined.sum() != 0]

# reorder columns
epa_combined = epa_combined[['CensusTract', 'Year', 'State', 'County'] + [col for col in epa_combined.columns if col not in ['CensusTract', 'Year', 'State', 'County']]]

# print to csv
epa_combined.to_csv('data/epa-airtoxscreen/epa_combined.csv', index=False)

In [42]:
# Group by census tract and year, then aggregate the pollutants column
grouped = epa_combined.groupby(['CensusTract', 'Year'])['Pollutant Name'].apply(lambda x: ' | '.join(x)).reset_index()

# Rename the aggregated column to 'Combined Pollutants'
grouped = grouped.rename(columns={'Pollutant Name': 'Combined Pollutants'})

# Merge the grouped DataFrame back to the original DataFrame
test = pd.merge(epa_combined, grouped, on=['CensusTract', 'Year'], how='left')

# drop original pollutants column
test = test.drop(columns='Pollutant Name')
#reorder columns
test = test[['CensusTract', 'Year', 'State', 'County', 'Population', 'Combined Pollutants'] + [col for col in test.columns if col not in ['CensusTract', 'Year', 'State', 'County', 'Population','Combined Pollutants']]]
#print to csv
test.to_csv('data/epa-airtoxscreen/test.csv', index=False)

In [32]:
### Social Vulnerability Index Data Cleaning ###

# read data
social_2016 = pd.read_csv('data/social-vulnerability-index/Utah-2016.csv')
social_2018 = pd.read_csv('data/social-vulnerability-index/Utah-2018.csv')
social_2020 = pd.read_csv('data/social-vulnerability-index/Utah-2020.csv')

# add 'Year' column
social_2016.insert(0, 'Year', 2016)
social_2018.insert(0, 'Year', 2018)
social_2020.insert(0, 'Year', 2020)

# Concatenate DataFrames for all years
svi_combined = pd.concat([social_2016, social_2018, social_2020], ignore_index=True)

# format columns
svi_combined = svi_combined.drop(columns=['ST','STATE','STCNTY',"LOCATION"])
svi_combined = svi_combined.rename(columns={'FIPS':'CensusTract','COUNTY':'County','ST_ABBR':'State'})

# reorder columns
svi_combined = svi_combined[['CensusTract', 'Year', 'State', 'County'] + [col for col in svi_combined.columns if col not in ['CensusTract', 'Year', 'State', 'County']]]


In [33]:
### Food access data cleaning ###

# read data
food_2010 = pd.read_csv('data/USDA_food_access/food_access_2010.csv')
food_2015 = pd.read_csv('data/USDA_food_access/food_access_2015.csv')
food_2019 = pd.read_csv('data/USDA_food_access/food_access_2019.csv')

# add 'Year' column
food_2010.insert(0, 'Year', 2010)
food_2015.insert(0, 'Year', 2015)
food_2019.insert(0, 'Year', 2019)

# Concatenate DataFrames for all years
food_combined = pd.concat([food_2010, food_2015, food_2019], ignore_index=True)

# reorder columns
food_combined = food_combined[['CensusTract', 'Year', 'State', 'County'] + [col for col in food_combined.columns if col not in ['CensusTract', 'Year', 'State', 'County']]]


In [34]:
# Merge all data
merged_data = pd.merge(epa_combined, svi_combined, on=['CensusTract','Year','State','County'], how='outer')
merged_data = pd.merge(merged_data, food_combined, on=['CensusTract','Year','State','County'], how='outer')
merged_data = merged_data.drop_duplicates()

# print to csv
merged_data.to_csv('data/merged_data.csv', index=False)